In [39]:
import pandas as pd

from copy import deepcopy
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler

import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras.regularizers import l1, l2

import tensorflow as tf
from tensorflow.keras import backend as K

import yfinance as yf

from sklearn import tree
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")


In [40]:
def construct_features_single_asset(df,k,h, linear = False):
    df = df.dropna(how='any',axis=0) 
    df['Cummulative Return'] = (1+ df['Return Daily']).cumprod(axis = 0)
    df['Next H Return'] = df['Cummulative Return'].pct_change(h).shift(-h)
    df['Mean H Return'] = df["Return Daily"].rolling(h).apply(lambda x: x.iloc[range(0,h)].mean()).shift(-h + 1)
    df['Square Sum Return'] = df["Return Daily"].rolling(h).apply(lambda x: x.iloc[range(0,h)].pow(2).sum()).shift(-h + 1)
    df['STD H Return'] = df["Return Daily"].rolling(h).apply(lambda x: x.iloc[range(0,h)].std(ddof = 1)).shift(-h + 1)

    for temp in range(k,0,-1):
        df["Before " + str(temp) + " Day" ] = df['Return Daily'].shift(periods = int(temp))

    if linear == True:
        df['Signal'] = [1 if x > 0 else -1 for x in df['Mean H Return']]

    df = df.dropna(how='any',axis=0)
    df = df[1:]

    return df

In [41]:
def feature_engineering(data,k,h,linear = False):
    company = data.columns
    features = []
    for i in range(k,0,-1):
        features.append("Before " + str(i) + " Day")
    features.append("Return Daily")

    X_train = pd.DataFrame(columns=features)
    if linear == False:
        y_train = pd.DataFrame(columns=["Mean H Return","Square Sum Return"])
    elif linear == True:
        y_train = pd.DataFrame(columns=["Signal"])
    for oo in company:
        flag_h = 0
        flag_k = k+1
        df = data[[oo]].copy()
        
        df.columns = ["Return Daily"]

        df = construct_features_single_asset(df,k,h,linear = linear)
        
        X_train = pd.concat([X_train,df[features]],axis = 0)
        if linear == False:
            y_train = pd.concat([y_train,df[["Mean H Return","Square Sum Return"]]],axis = 0)
        elif linear == True:
            y_train = pd.concat([y_train,df[["Signal"]]],axis = 0)
    return [X_train,y_train]

In [ ]:
def sharpe_loss(h):
    def calculation(y_target, y_pred):

        mean = K.reshape(y_target[:, 0], (-1, 1))
        square_sum =  K.reshape(y_target[:, 1], (-1, 1))

        sum_pofolio = mean * h * y_pred
        mean_pofolio = K.mean(mean * h * y_pred)

        std_pofolio = tf.math.sqrt(K.mean(square_sum - 2 * sum_pofolio * mean_pofolio + mean_pofolio ** 2)/h)

    
        return  - (mean_pofolio / std_pofolio) *np.sqrt(252)
    
    return calculation

In [ ]:
def train_decision_tree(data, k, h):

    model = tree.DecisionTreeClassifier()

    X_train,y_train = feature_engineering(data,k,h,linear = True)
    X_train = np.array(X_train, dtype=np.float64)
    y_train = np.array(y_train, dtype=np.float64)
    model= model.fit(X_train, y_train)
    
    return model

In [ ]:
def train_xgboost(data, k, h):

    model = xgb.XGBRegressor(objective="multi:softmax", num_class = 2,random_state=42)

    X_train,y_train = feature_engineering(data,k,h,linear = True)
    X_train = np.array(X_train, dtype=np.float64)
    y_train = np.array(y_train, dtype=np.float64)
    y_train[y_train == -1] = 0
    model= model.fit(X_train, y_train)
    
    return model

In [ ]:
def train_MLP_supervised(data, k, h):

    model = Sequential([
    Dropout(0, input_shape=(k+1,)),
    Dense(20,activation = 'tanh'),
    Dense(1,activation = 'tanh'),
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy')


    X_train,y_train = feature_engineering(data,k,h,linear = True)
    X_train = np.array(X_train, dtype=np.float64)
    y_train = np.array(y_train, dtype=np.float64)
    model.fit(X_train, y_train, epochs=100, batch_size = 256, verbose=1)
    
    return model

In [ ]:
def train_lasso_supervised(data, k, h,lambda_val = .7):

    model = Sequential([
        Dense(1, input_shape = (k+1,),kernel_regularizer = l1(lambda_val))
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy')


    X_train,y_train = feature_engineering(data,k,h,linear = True)
    X_train = np.array(X_train, dtype=np.float64)
    y_train = np.array(y_train, dtype=np.float64)
    model.fit(X_train, y_train, epochs=100, batch_size = 256, verbose=1)
    
    return model

In [43]:
def train_MLP_sharpeLoss(data, k, h):

    model = Sequential([
    Dropout(0, input_shape=(k+1,)),
    Dense(20,activation = 'tanh'),
    Dense(1,activation = 'tanh'),
    ])

    model.compile(optimizer='adam', loss=sharpe_loss(h = h))


    X_train,y_train = feature_engineering(data,k,h,linear = False)
    
    model.fit(X_train, y_train, epochs=100, batch_size = 256, verbose=1)
    
    return model

In [64]:
def train_Lasso_sharpeLoss(data, k, h,lambda_val = .01):

    model = Sequential([
        Dense(1, input_shape = (k+1,),kernel_regularizer = l1(lambda_val))
    ])

    model.compile(optimizer='adam', loss=sharpe_loss(h = h))

    X_train,y_train = feature_engineering(data,k,h,linear = False)

    model.fit(X_train, y_train, epochs=50, batch_size = 1024, verbose=1)
    
    return model

In [ ]:
time_range = '6mo' # khoảng thời gian làm backtest 
start_time = '2024-01-01'
end_time = '2024-07-01'

def EU_Stock_data():
    """Lấy dữ liệu giá Close của 50 công ty trên sàn Euro_STOXX 50 vào thời gian cho trước"""

    stock_list = pd.read_html( 'https://en.wikipedia.org/wiki/EURO_STOXX_50')[4]['Ticker'][1:].to_list()
    futures = pd.DataFrame(columns= stock_list) # danh sách mã
    
    # đặt index 
    time_index = list(yf.Ticker(stock_list[0]).history(period = time_range,start = start_time, end = end_time).index) 

    # xét từng mã
    for symbol in stock_list:
        df = yf.Ticker(symbol).history(period = time_range, start = start_time, end = end_time)
        df = pd.DataFrame(df['Close']) # lấy giá close
        i = 0
        daily_return = []
        # tinh daily return, = 0 trong ngày đầu tiên 
        for k in df['Close']:
            if i != 0:
                daily_return.append(float((k-i)/i))
            else:
                daily_return.append(float(0))
            i = k
        try:
            futures[symbol] = daily_return
        except:
            while len(daily_return) < len(futures):
                daily_return.insert(0,np.nan)
            futures[symbol] = daily_return

    futures.index = time_index

    futures['Date'] = pd.to_datetime(futures.index, format='%Y-%m-%d')
    futures.set_index('Date', inplace=True)

    return futures

In [ ]:
def test_model_TSMOM(data, model,k,h,linear = False):

    company = data.columns

    signal = pd.DataFrame(index = data.index, columns= company)

    features = []
    for i in range(k,0,-1):
        features.append("Before " + str(i) + " Day")
    features.append("Return Daily")

    for oo in company:
        df = data[[oo]].copy()
        
        df.columns = ["Return Daily"]
        df = construct_features_single_asset(df,k,h,linear = linear)

        
        X_test = df[features]
        X_test['prediction'] = np.sign(model.predict(X_test))
        X_test['prediction'][X_test['prediction'] == 0] = -1
        for x,v in enumerate(X_test.index):
            signal.loc[v,oo] = X_test.loc[v,'prediction']
        
        signal[oo] = signal[oo].ffill()
        signal[oo] = signal[oo].fillna(0)


    return signal

In [ ]:
def Volatility_scale(data, ignore_na=False, adjust = True, com = 60, min_periods=0):
    """Scale data using ex ante volatility"""

    # Lưu trữ index, tức thời gian 
    std_index = data.index

    # chứa kết quả
    daily_index = pd.DataFrame(index=std_index)

    # xét từng cổ phiếu
    for oo in data.columns:
        returns = data[oo]  # Lấy ra các return
        returns.dropna(inplace=True)  # xử lý null bằng zero

        # Tính cumulative (cum) return , nhưng ko có thành phần - 1
        ret_index = (1 + returns).cumprod()

        # Tính daily volatility (vol)
        day_vol = returns.ewm(ignore_na=ignore_na,
                              adjust=adjust,
                              com=com,
                              min_periods=min_periods).std(bias=False)
        
        vol = day_vol * np.sqrt(252)  # scale lại theo 252 ngày active trading

        # Join cum return và vol
        ret_index = pd.concat([ret_index, vol], axis=1)
        ret_index.columns = [oo, oo + '_Vol']  # Đặt tên cột cum return là tên cổ phiếu, bên cạnh là vol 

        # Join 
        daily_index = pd.concat([daily_index, ret_index], axis=1)

    return daily_index


In [ ]:
def classic_TSMOM(data, k, h, tolerance = 0,ignore_na = False, adjust = True, com = 60, min_periods = 0):
    signal = pd.DataFrame(index = data.index)

    # gọi hàm Volatility scale
    daily_index = Volatility_scale(data,ignore_na=ignore_na,
                          adjust=adjust,
                          com=com,   
                          min_periods = min_periods)

    company = data.columns

    for oo in company:
        flag_h = 0
        flag_k = k+1
        df = pd.concat([daily_index[oo], daily_index[oo+"_Vol"]], axis=1)
        df['rolling returns'] = df[oo].pct_change(k) # so sánh thay đổi ở ngày t với k ngày trước đó (tức t - k)
        df['signal'] = 0.
        for x, v in enumerate(df['rolling returns']):
            if flag_h != 0:
                # Bỏ qua giai đoạn hold, tránh bị tính lặp lại
                flag_h = flag_h - 1
                continue
            # Bỏ qua thời gian cty chưa được lên sàn (nêu có)
            if df[oo].isnull().iloc[x] == False:
                # bỏ qua k ngày đầu vì chưa đủ k lookback
                if flag_k != 0:
                    flag_k = flag_k - 1
                    continue
            else: continue
            try:
                if df['rolling returns'].iloc[x-1] < tolerance:
                    for h_period in range(0,h):
                        # rolling return < 0, short rồi giữ trong h ngày, tính pnl, leverage///
                        df['signal'].iloc[x + h_period] = -1
                
                elif df['rolling returns'].iloc[x-1] > tolerance:
                    for h_period in range(0,h):
                        # rolling return > 0, long rồi giữ trong h ngày, tính pnl, leverage///
                        df['signal'].iloc[x + h_period] = 1

            except:pass
            

            # Đặt flag holding là h - 1, để qua vòng for mới bỏ qua ngày hold, tránh bị tính lặp lại
            if df['rolling returns'].iloc[x-1] != tolerance: flag_h = h - 1

        signal = pd.concat([signal, df['signal']], axis=1)

    signal.columns = data.columns
    
    return signal

In [ ]:
def backtest(data,signal,k,h,  vol_flag = 1, target_vol = 0.2, ignore_na = False, adjust = True, com = 60, min_periods = 0):
    
    pnl = pd.DataFrame(index=data.index)
    leverage = pd.DataFrame(index = data.index)

    # gọi hàm Volatility scale
    daily_index = Volatility_scale(data,ignore_na=ignore_na,
                          adjust=adjust,
                          com=com,   
                          min_periods = min_periods)

    company = data.columns

    # Volatility settings
    vol_flag = vol_flag    # Set flag to 1 for vol targeting
    if vol_flag == 1:
        target_vol = target_vol 
    else:
        target_vol = 'no target vol'
    

    for oo in company:
        flag_h = 0
        flag_k = k+1
        df = pd.concat([daily_index[oo], daily_index[oo+"_Vol"]], axis=1)

        df['pnl'] = 0. 
        df['leverage'] = 0.
        for x, v in enumerate(df['pnl']):
            if flag_h != 0:
                # Bỏ qua giai đoạn hold, tránh bị tính lặp lại
                flag_h = flag_h - 1
                continue
            # Bỏ qua thời gian cty chưa được lên sàn (nêu có)
            if df[oo].isnull().iloc[x] == False:
                # bỏ qua k ngày đầu vì chưa đủ k lookback
                if flag_k != 0:
                    flag_k = flag_k - 1
                    continue
            else: continue
            try:
                if signal[oo].iloc[x] == -1:
                    for h_period in range(0,h):
                        if vol_flag == 1:
                            df['pnl'].iloc[x + h_period] = (1 - df[oo].iloc[x + h_period] / df[oo].iloc[x - 1 + h_period]) * \
                                target_vol / df[oo+"_Vol"].iloc[x -1] 
                            df['leverage'].iloc[x + h_period] = target_vol / df[oo+"_Vol"].iloc[x -1]
                        else:
                            df['pnl'].iloc[x + h_period] = (1 - df[oo].iloc[x + h_period] / df[oo].iloc[x - 1 + h_period])
                            df['leverage'].iloc[x+h_period] = 1
                elif signal[oo].iloc[x] == 1:
                    for h_period in range(0,h):
                        if vol_flag == 1:
                            df['pnl'].iloc[x + h_period] = (df[oo].iloc[x + h_period] / df[oo].iloc[x - 1 + h_period] - 1) * \
                                    target_vol / df[oo+"_Vol"].iloc[x - 1]
                            df['leverage'].iloc[x+h_period] = target_vol / df[oo+"_Vol"].iloc[x -1]
                        else:
                            df['pnl'].iloc[x + h_period] = (df[oo].iloc[x + h_period] / df[oo].iloc[x - 1 + h_period] - 1)
                            df['leverage'].iloc[x+h_period] = 1
            except:pass
            
            if signal[oo].iloc[x] == 1 or signal[oo].iloc[x] == -1 : flag_h = h - 1


        leverage = pd.concat([leverage, df['leverage']], axis = 1)
        pnl = pd.concat([pnl, df['pnl']], axis=1)

    pnl.columns = data.columns
    leverage.columns = data.columns

    return [pnl,leverage]

In [ ]:
def strategy_daily_return(pnl):
    
    return pnl.mean(skipna = False, axis=1)

In [ ]:
train_data = pd.read_csv('Main File/TSMOM/Data/data_close.csv' , index_col= 'Date')
test_data = EU_Stock_data()

k = 10
h = 15

model_name = ['classic_TSMOM','train_decision_tree','train_xgboost','train_MLP_supervised','train_lasso_supervised','train_MLP_sharpeLoss','train_Lasso_sharpeLoss']

for model in model_name:
    func = globals()[model]
    if model == 'classic_TSMOM':
        pnl = strategy_daily_return(backtest(test_data,func(test_data,k,h),k,h)[0])
    else:
        temp_model = func(train_data,10,15)
        pnl = strategy_daily_return(backtest(test_data,test_model_TSMOM(test_data,temp_model,k,h),k,h)[0])
    try:
        temp = pnl.to_list()
        temp.insert(0,model)
        stats.loc[len(stats.index)] = temp
    except:
        index = pnl.index.to_list()
        index.insert(0,'Model')
        stats = pd.DataFrame(columns = index)
        temp = pnl.to_list()
        temp.insert(0,model)
        stats.loc[len(stats.index)] = temp

Epoch 1/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.0350  
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.3543 
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.7185 
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -1.0113 
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: -1.3798
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -1.6053 
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -2.1160 
Epoch 8/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -2.3767 
Epoch 9/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: -2.7391
Epoch 10/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -3.0139 
Epoch 11/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -3.4369 
Epoch 12/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -3.4282 
Epoch 13/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -3.9787 
Epoch 14/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: -4.3475
Epoch 15/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 

In [66]:
stats

,Model,2024-01-02 00:00:00+01:00,2024-01-03 00:00:00+01:00,2024-01-04 00:00:00+01:00,2024-01-05 00:00:00+01:00,2024-01-08 00:00:00+01:00,2024-01-09 00:00:00+01:00,2024-01-10 00:00:00+01:00,2024-01-11 00:00:00+01:00,2024-01-12 00:00:00+01:00,...,2024-06-17 00:00:00+02:00,2024-06-18 00:00:00+02:00,2024-06-19 00:00:00+02:00,2024-06-20 00:00:00+02:00,2024-06-21 00:00:00+02:00,2024-06-24 00:00:00+02:00,2024-06-25 00:00:00+02:00,2024-06-26 00:00:00+02:00,2024-06-27 00:00:00+02:00,2024-06-28 00:00:00+02:00
0,classic_TSMOM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.006402,-0.004838,0.002777,-0.006243,0.003719,-0.009205,0.004129,0.004359,0.003279,0.002627
1,train_decision_tree,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001823,-0.000835,-0.000044,-0.001196,0.000747,0.001468,0.001998,0.001791,0.000201,0.001989
2,train_xgboost,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001370,-0.000441,-0.001442,0.000140,0.000356,0.000074,0.000421,0.002667,0.000301,-0.000575
3,train_MLP_supervised,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005564,0.004349,-0.001691,0.007339,-0.004651,0.009929,0.001411,-0.004248,-0.003863,-0.001750
4,train_lasso_supervised,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.006410,0.005911,-0.002360,0.008773,-0.004927,0.011131,-0.003775,-0.005797,-0.004686,-0.002447
5,train_MLP_sharpeLoss,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002897,0.001626,-0.000833,0.002939,-0.001498,0.005715,0.001005,-0.000600,-0.000656,-0.000901
6,train_Lasso_sharpeLoss,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,train_Lasso_sharpeLoss,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.006410,0.005911,-0.002360,0.008773,-0.004927,0.011131,-0.003775,-0.005797,-0.004686,-0.002447


In [70]:
stats = stats.reset_index(drop = True)
stats

,Model,2024-01-02 00:00:00+01:00,2024-01-03 00:00:00+01:00,2024-01-04 00:00:00+01:00,2024-01-05 00:00:00+01:00,2024-01-08 00:00:00+01:00,2024-01-09 00:00:00+01:00,2024-01-10 00:00:00+01:00,2024-01-11 00:00:00+01:00,2024-01-12 00:00:00+01:00,...,2024-06-17 00:00:00+02:00,2024-06-18 00:00:00+02:00,2024-06-19 00:00:00+02:00,2024-06-20 00:00:00+02:00,2024-06-21 00:00:00+02:00,2024-06-24 00:00:00+02:00,2024-06-25 00:00:00+02:00,2024-06-26 00:00:00+02:00,2024-06-27 00:00:00+02:00,2024-06-28 00:00:00+02:00
0,classic_TSMOM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.006402,-0.004838,0.002777,-0.006243,0.003719,-0.009205,0.004129,0.004359,0.003279,0.002627
1,train_decision_tree,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001823,-0.000835,-0.000044,-0.001196,0.000747,0.001468,0.001998,0.001791,0.000201,0.001989
2,train_xgboost,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001370,-0.000441,-0.001442,0.000140,0.000356,0.000074,0.000421,0.002667,0.000301,-0.000575
3,train_MLP_supervised,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005564,0.004349,-0.001691,0.007339,-0.004651,0.009929,0.001411,-0.004248,-0.003863,-0.001750
4,train_lasso_supervised,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.006410,0.005911,-0.002360,0.008773,-0.004927,0.011131,-0.003775,-0.005797,-0.004686,-0.002447
5,train_MLP_sharpeLoss,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002897,0.001626,-0.000833,0.002939,-0.001498,0.005715,0.001005,-0.000600,-0.000656,-0.000901
6,train_Lasso_sharpeLoss,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.006410,0.005911,-0.002360,0.008773,-0.004927,0.011131,-0.003775,-0.005797,-0.004686,-0.002447


In [71]:
stats.to_csv("result.csv")